In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
from tqdm import tqdm
import json

inpath= path='' #input image path (Json files also will be saved here default)
modelpath=''    #Yolo model path    
savepath=''     #save segment images for visualisation(optional)

colors=[(235, 0, 167), (222, 107, 31), (238, 187, 137), (121, 223, 84), (191, 160, 34), (185, 120, 125), (106, 104, 198), (24, 101, 136), 
        (121, 243, 36), (79, 137, 147), (240, 75, 25), (202, 102, 95), (249, 7, 93), (164, 82, 70), (124, 113, 209), (135, 117, 98), 
        (93, 159, 121), (103, 4, 88), (115, 45, 100), (120, 69, 165), (78, 141, 239), (203, 12, 150), (85, 222, 142), (213, 243, 50), 
        (123, 197, 89), (212, 4, 193), (130, 251, 249), (55, 61, 177), (255, 76, 218), (150, 150, 141), (14, 48, 57), (45, 63, 177), 
        (231, 147, 222), (72, 20, 99), (185, 123, 202), (155, 189, 220), (221, 151, 19), (63, 240, 92), (245, 21, 137), (105, 190, 151), 
        (163, 170, 59), (5, 90, 87), (94, 167, 253), (4, 227, 232), (9, 53, 228), (56, 214, 209), (250, 224, 251), (127, 229, 161), 
        (177, 152, 5), (230, 243, 190), (239, 107, 12)]
model_seg = YOLO(modelpath) 
for ifile in tqdm(sorted(os.listdir(inpath))[:]): 
    try:
        if not (ifile.endswith('.jpg') or ifile.endswith('.JPG') or ifile.endswith('.png')):
            continue
        results_mask=model_seg.predict(path+ifile,conf=0.2,imgsz=2016,iou=0.3,classes=[3,4,6],verbose=False) 
        lnames=list(model_seg.names.values())
        image=cv2.imread(path+ifile)
        h,w,_=image.shape
        image2=image.copy()
        for result in results_mask:
            segments=result.masks.xy
            bboxes = [bbox.cpu().numpy() for bbox in result.boxes.xyxy]
            confi = [conf.item() for conf in result.boxes.conf]
            cls = [int(x.item()) for x in result.boxes.cls]
            area=[cv2.contourArea(x) for x in result.masks.xy]
            jdata={
                "version": "5.4.1",
                "flags": {},
                "shapes":[],
                "imagePath": ifile,
                "imageData": None,
                "imageHeight": h,
                "imageWidth": w
            }
            for seg2,cls2,confi2,boxes2 in zip(segments,cls,confi,bboxes):
                text=lnames[cls2]
                color=colors[lnames.index(lnames[cls2])]
                x1, y1, x2, y2 = boxes2
                if len(seg2)<4:
                    continue
                segdata={
                    "label":text,
                    "points":[list(map(float, s)) for s in seg2],
                    "group_id": None,
                    "description": "",
                    "shape_type": "polygon",
                    "flags": {},
                    "mask": None
                    
                }
                jdata['shapes'].append(segdata)

            ############ Uncomment below portion for visualising the output segmentation ######################
            #     cv2.fillPoly(image, pts=[seg2.astype('int32')], color=color)
            #     cv2.rectangle(image2, (int(x1), int(y1)), (int(x2), int(y2)), color, 1)
            #     cv2.putText(image2, text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1)
            #     img3 = cv2.addWeighted(image, 0.5, image2, 0.5,5)
            # cv2.imwrite(savepath+ifile,img3) 

            with open(inpath+ifile[:-4]+'.json','w') as js:
                json.dump(jdata,js,indent = 4)
    except Exception as e:
        print(e)
        pass